# Storage googleapis

In [1]:
import os

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Reshape
from tensorflow.keras.layers.experimental.preprocessing import Resizing

!wget -q https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
!unzip -q cats_and_dogs_filtered.zip

TRAIN_DIR = "./cats_and_dogs_filtered/train"
VAL_DIR = "./cats_and_dogs_filtered/validation"

IMG_SIZE = (224, 224)
INP_SHAPE = (*IMG_SIZE, 3)
BATCH_SIZE = 64

def gen_new_data(data_folder, augmented=True):
  if augmented:
    data_gen = ImageDataGenerator(rescale=1./255, rotation_range=40,
                                  width_shift_range=0.2, height_shift_range=0.2,
                                  horizontal_flip=True)
  else:
    data_gen = ImageDataGenerator(rescale=1./255)

  data = data_gen.flow_from_directory(
      data_folder,
      target_size=IMG_SIZE,
      batch_size=BATCH_SIZE,
      class_mode="binary",
  )
  return data

ds_train = gen_new_data(TRAIN_DIR, augmented=True)
ds_val = gen_new_data(VAL_DIR, augmented=False)

mobilenet_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4" 

mobilenet = Sequential()
mobilenet.add(Resizing(224, 224, input_shape=INP_SHAPE))
mobilenet.add(hub.KerasLayer(mobilenet_url, trainable=False))
mobilenet.add(Dense(1, activation="sigmoid"))

mobilenet.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

mobilenet.fit(ds_train, validation_data=ds_val,
              batch_size=32, epochs=10,verbose=1)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/10
32/32 [==============================] - 36s 858ms/step - loss: 0.4047 - accuracy: 0.8225 - val_loss: 0.1143 - val_accuracy: 0.9740
Epoch 2/10
32/32 [==============================] - 27s 846ms/step - loss: 0.1495 - accuracy: 0.9465 - val_loss: 0.0799 - val_accuracy: 0.9810
Epoch 3/10
32/32 [==============================] - 27s 844ms/step - loss: 0.1075 - accuracy: 0.9656 - val_loss: 0.0653 - val_accuracy: 0.9830
Epoch 4/10
32/32 [==============================] - 27s 844ms/step - loss: 0.0951 - accuracy: 0.9690 - val_loss: 0.0600 - val_accuracy: 0.9830
Epoch 5/10
32/32 [==============================] - 27s 843ms/step - loss: 0.0862 - accuracy: 0.9714 - val_loss: 0.0555 - val_accuracy: 0.9830
Epoch 6/10
32/32 [==============================] - 27s 845ms/step - loss: 0.0935 - accuracy: 0.9655 - val_loss: 0.0530 - val_accuracy: 0.9840
Epoch 7/10
32/32 [==============================] - 27s 83

# DATASETS

In [17]:
from sklearn.preprocessing import minmax_scale

In [5]:
ds_train, ds_info = tfds.load(
    'cats_vs_dogs',
    split=tfds.Split.TRAIN,
    with_info=True,
)

In [49]:
num_class = 2
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset = -1)

def preprocess(features):
  
    image = tf.image.resize(features['image'], [224, 224])
    image = tf.cast(image, tf.float32)

    #label = tf.one_hot(features['label'], depth=num_class)
    label = features['label']
    return image, label

def solution_model():

  train_dataset = ds_train.map(preprocess).batch(32)
  train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))

  mobilenet_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4" 
  model = Sequential()
  
  model.add(hub.KerasLayer(mobilenet_url, trainable=False, input_shape=(224, 224, 3)))
  model.add(Dense(1, activation="sigmoid"))

  model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

  model.fit(train_dataset,
              batch_size=32, epochs=10,verbose=1)
  
  return model


In [50]:
model = solution_model()

Epoch 1/10
727/727 [==============================] - 36s 45ms/step - loss: 0.2643 - accuracy: 0.8806
Epoch 2/10
727/727 [==============================] - 33s 46ms/step - loss: 0.1281 - accuracy: 0.9488
Epoch 3/10
727/727 [==============================] - 34s 46ms/step - loss: 0.1193 - accuracy: 0.9528
Epoch 4/10
727/727 [==============================] - 34s 46ms/step - loss: 0.1138 - accuracy: 0.9544
Epoch 5/10
727/727 [==============================] - 33s 46ms/step - loss: 0.1097 - accuracy: 0.9558
Epoch 6/10
727/727 [==============================] - 34s 46ms/step - loss: 0.1065 - accuracy: 0.9569
Epoch 7/10
727/727 [==============================] - 33s 45ms/step - loss: 0.1038 - accuracy: 0.9580
Epoch 8/10
727/727 [==============================] - 33s 46ms/step - loss: 0.1016 - accuracy: 0.9591
Epoch 9/10
727/727 [==============================] - 34s 47ms/step - loss: 0.0997 - accuracy: 0.9596
Epoch 10/10
727/727 [==============================] - 34s 46ms/step - loss: 0.098